# Data generator
This notebook uses the simulated and real datasets to produce the generated images using 3 pix2pix and 3 CycleGAN networks.
It aslo produces the segmentation masks using Segformer.

## Usage:
### task_type='donkey' <- Choose task domain 'donkey' or 'kitti' for lane-keeping or vehicle detection
Run all cells

## Requirements:
-Segformer checkpoints for segmentation: \
    ./[task_type]/content/segmentation_checkpoints/Model_weights.hdf5 \
    
    Please refer to specific [task_type] to generate segmentation checkpoint

-CycleGAN and pix2pix checkpoints for I2I translation:\
    ./[task_type]/content/gan_checkpoints/cyclegan_checkpoints/1/ \
    ./[task_type]/content/gan_checkpoints/cyclegan_checkpoints/2/ \
    ./[task_type]/content/gan_checkpoints/cyclegan_checkpoints/3/ \
    ./[task_type]/content/gan_checkpoints/pix2pix_checkpoints/1/ \
    ./[task_type]/content/gan_checkpoints/pix2pix_checkpoints/2/ \
    ./[task_type]/content/gan_checkpoints/pix2pix_checkpoints/3/ 

    Please refer to specific [task_type] to generate GAN checkpoints

-Dataset h5 files: \
    KITTI: \
    ./[task_type]/content/datasets/h5_out/bounding_boxes_real.h5\
    ./[task_type]/content/datasets/h5_out/bounding_boxes_sim.h5\
    ./[task_type]/content/datasets/h5_out/raw_image_real.h5\
    ./[task_type]/content/datasets/h5_out/raw_image_sim.h5\
    ./[task_type]/content/datasets/h5_out/segmentation_masks_real.h5\
    ./[task_type]/content/datasets/h5_out/segmentation_masks_sim.h5\
    ./[task_type]/content/datasets/h5_out/semantic_id_list_real.h5\
    ./[task_type]/content/datasets/h5_out/semantic_id_list_sim.h5\
    DONKEY:\
    ./[task_type]/content/datasets/h5_out/gt_real.h5\
    ./[task_type]/content/datasets/h5_out/raw_image_real.h5\
    ./[task_type]/content/datasets/h5_out/raw_image_sim.h5\
    ./[task_type]/content/datasets/h5_out/semantic_id_list_real.h5\
    ./[task_type]/content/datasets/h5_out/semantic_id_list_sim.h5\

    Please refer to specific [task_type] to generate h5 files

## Outputs:
    ./[task_type]/content/output_plots/[domain_type]/[domain_type] 
    ./[task_type]/content/output_plots/[domain_type]/[domain_type]_mask 
    ./[task_type]/content/output_plots/[domain_type]/[domain_type]_additional_mask 
    ./[task_type]/content/output_plots/[domain_type]/[domain_type]_mask_error


In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf

import os
import matplotlib.pyplot as plt
import json
import tensorflow as tf
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix
import os
import matplotlib.pyplot as plt

import data_generator_utils


OUTPUT_CHANNELS = 3
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 512
IMG_HEIGHT = 512

%load_ext autoreload
%autoreload 2

In [2]:
def save_sim_real_outputs(
    task_type,
    segmentation_model,
    real_path,
    sim_path,
    additional_id,
    height,
    width,
    dataset_index_list_test,
    test_indexes_gan,
    loaded_dictionary_images_real,
    loaded_dictionary_images_sim
):
    for dataset_index in dataset_index_list_test:
          print(dataset_index)
          counter=0
          for i in test_indexes_gan[dataset_index]:
                if i%50==0:
                    print(i," out of ",len(test_indexes_gan[dataset_index]))
                counter+=1
                input_image_real = np.array(loaded_dictionary_images_real[dataset_index][i])
                input_image_real = tf.convert_to_tensor(input_image_real.astype(np.uint8), np.uint8)
                input_image_real = tf.reverse(input_image_real, axis=[-1])
                
                png_image = tf.image.encode_png(input_image_real)
                os.makedirs('./'+task_type+'/content/output_plots/real/'+real_path+'/', exist_ok=True)
                with open('./'+task_type+'/content/output_plots/real/'+real_path+'/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())
                input_image_real = tf.cast(input_image_real, np.float16)
                input_image_real = tf.transpose(input_image_real, (2, 0, 1))
        
                input_image_sim = np.array(loaded_dictionary_images_sim[dataset_index][i])
                input_image_sim_for_pix2pix=input_image_sim
                input_image_sim = tf.convert_to_tensor(input_image_sim.astype(np.uint8), np.uint8)
                input_image_sim = tf.reverse(input_image_sim, axis=[-1])
                png_image = tf.image.encode_png(tf.cast(input_image_sim,np.uint8))
                os.makedirs('./'+task_type+'/content/output_plots/sim/'+sim_path+'/', exist_ok=True)
                with open('./'+task_type+'/content/output_plots/sim/'+sim_path+'/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())
                input_image_sim = tf.cast(input_image_sim, np.float16)
                input_image_sim = tf.transpose(input_image_sim, (2, 0, 1))
                
                colored_mask = np.zeros_like(input_image_sim_for_pix2pix)
                input_image_real=tf.expand_dims(input_image_real, 0)
                
                pred_masks =segmentation_model.predict(input_image_real).logits
                created_mask=data_generator_utils.cast_to_int32(data_generator_utils.create_mask(pred_masks))
                created_mask_real = tf.image.resize(created_mask, [height,width],
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                created_mask_real = tf.cast(created_mask_real, np.uint8)

                input_image_sim=tf.expand_dims(input_image_sim, 0)
                pred_masks =segmentation_model.predict(input_image_sim).logits
                created_mask=data_generator_utils.cast_to_int32(data_generator_utils.create_mask(pred_masks))
                created_mask_sim = tf.image.resize(created_mask, [height,width],
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                created_mask_sim = tf.cast(created_mask_sim, np.uint8)

                if task_type=="kitti":
                    vectorized_map_additional=np.vectorize(data_generator_utils.map_values_car)
                    
                elif task_type=="donkey":
                    vectorized_map_additional=np.vectorize(data_generator_utils.map_values_donkey)

                real_mask=vectorized_map_additional(created_mask_real)
                sim_mask=vectorized_map_additional(created_mask_sim)
              
                colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(created_mask_real))
                colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(created_mask_real))
                colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(created_mask_real))
                png_image = tf.image.encode_png(colored_mask)
                os.makedirs('./'+task_type+'/content/output_plots/real/'+real_path+'_mask/', exist_ok=True)
                with open('./'+task_type+'/content/output_plots/real/'+real_path+'_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())

                if task_type=="donkey":
                    binary_mask_real1 = (real_mask == 1).astype(np.uint8)
                    binary_mask_real2 = (real_mask == 2).astype(np.uint8)
                    binary_mask_real = binary_mask_real1+binary_mask_real2
                else:
                    binary_mask_real = (real_mask == additional_id).astype(np.uint8)

                colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(binary_mask_real))
                colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(binary_mask_real))
                colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(binary_mask_real))
                png_image = tf.image.encode_png(colored_mask)
                os.makedirs('./'+task_type+'/content/output_plots/real/'+real_path+'_additional_mask/', exist_ok=True)
                with open('./'+task_type+'/content/output_plots/real/'+real_path+'_additional_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())

                colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(created_mask_sim))
                colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(created_mask_sim))
                colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(created_mask_sim))
                png_image = tf.image.encode_png(colored_mask)
                os.makedirs('./'+task_type+'/content/output_plots/sim/'+sim_path+'_mask/', exist_ok=True)
                with open('./'+task_type+'/content/output_plots/sim/'+sim_path+'_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())
              
                if task_type=="donkey":
                    binary_mask_sim1 = (sim_mask == 1).astype(np.uint8)
                    binary_mask_sim2 = (sim_mask == 2).astype(np.uint8)
                    binary_mask_sim = binary_mask_sim1+binary_mask_sim2
                else:
                    binary_mask_sim = (sim_mask == additional_id).astype(np.uint8)
                colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(binary_mask_sim))
                colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(binary_mask_sim))
                colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(binary_mask_sim))
                png_image = tf.image.encode_png(colored_mask)
                os.makedirs('./'+task_type+'/content/output_plots/sim/'+sim_path+'_additional_mask/', exist_ok=True)
                with open('./'+task_type+'/content/output_plots/sim/'+sim_path+'_additional_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())
                
                real_mask = np.array(real_mask)
                real_mask = tf.convert_to_tensor(real_mask.astype(np.float16), np.float16)
    
                sim_mask = np.array(sim_mask)
                sim_mask = tf.convert_to_tensor(sim_mask.astype(np.float16), np.float16)

                binary_mask_real = np.array(binary_mask_real)
                binary_mask_real = tf.convert_to_tensor(binary_mask_real.astype(np.float16), np.float16)

                binary_mask_sim = np.array(binary_mask_sim)
                binary_mask_sim = tf.convert_to_tensor(binary_mask_sim.astype(np.float16), np.float16)
    
                real_mask=data_generator_utils.cast_to_int32(real_mask)
                sim_mask=data_generator_utils.cast_to_int32(sim_mask)
                error_sim_real=abs(sim_mask-real_mask)
                error_sim_real_car=abs(binary_mask_sim-binary_mask_real)

                error_data_sim = {
                    "sim_real": np.sum(error_sim_real) / (height * width),
                    "additional": np.sum(error_sim_real_car) / (height * width)
                }

                output_folder = './'+task_type+'/content/output_plots/sim/'+sim_path+'_mask_error/'
                os.makedirs(output_folder, exist_ok=True)
                output_path = output_folder + str(dataset_index) + "_" + str(i) +".json"
                with open(output_path, "w") as json_file:
                    json.dump(error_data_sim, json_file, indent=4)

In [12]:
def save_cyclegan_outputs(
    task_type,
    segmentation_model,
    generator_cyclegan,
    cyclegan_name,
    additional_id,
    height,
    width,
    dataset_index_list_test,
    test_indexes_gan,
    loaded_dictionary_images_real,
    loaded_dictionary_images_sim
):
    for dataset_index in dataset_index_list_test:
          counter=0
          for i in test_indexes_gan[dataset_index]:
                if i%50==0:
                    print(i," out of ",len(test_indexes_gan[dataset_index]))
                counter+=1
                input_image_real = np.array(loaded_dictionary_images_real[dataset_index][i])
                input_image_real = tf.convert_to_tensor(input_image_real.astype(np.uint8), np.uint8)
                input_image_real = tf.reverse(input_image_real, axis=[-1])
                

                input_image_real = tf.cast(input_image_real, np.float16)
                input_image_real = tf.transpose(input_image_real, (2, 0, 1))
        
                input_image_sim = np.array(loaded_dictionary_images_sim[dataset_index][i])
                input_image_sim_for_pix2pix=input_image_sim
                input_image_sim = tf.convert_to_tensor(input_image_sim.astype(np.uint8), np.uint8)
                input_image_sim = tf.reverse(input_image_sim, axis=[-1])
                input_image_sim = tf.cast(input_image_sim, np.float16)
                input_image_sim = tf.transpose(input_image_sim, (2, 0, 1))
                
                vectorized_map_car=np.vectorize(data_generator_utils.map_values_car)
        
                input_image_sim_for_cyclegan=tf.expand_dims(input_image_sim, 0)
                input_image_sim_for_cyclegan={'pixel_values': input_image_sim_for_cyclegan }
              
                fake_real_cycle = data_generator_utils.generate_images_cyclegan(generator_cyclegan, input_image_sim_for_cyclegan,False,height,width)
                fake_real_cycle = tf.image.resize(fake_real_cycle, [height,width],
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                fake_real_cycle=fake_real_cycle * (255/2) + (255/2)
                fake_real_cycle = tf.cast(fake_real_cycle, np.uint8)
                png_image = tf.image.encode_png(fake_real_cycle)
                os.makedirs('./'+task_type+'/content/output_plots/cyclegan/'+cyclegan_name+'/', exist_ok=True)
                with open('./'+task_type+'/content/output_plots/cyclegan/'+cyclegan_name+'/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())
                fake_real_cycle = tf.cast(fake_real_cycle, np.float16)
                fake_real_cycle = tf.transpose(fake_real_cycle, (2, 0, 1))
              
                input_image_real=tf.expand_dims(input_image_real, 0)
                pred_masks =segmentation_model.predict(input_image_real).logits
                created_mask=data_generator_utils.cast_to_int32(data_generator_utils.create_mask(pred_masks))
                created_mask_real = tf.image.resize(created_mask, [height,width],
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                created_mask_real = tf.cast(created_mask_real, np.uint8)
                real_label_value =vectorized_map_car(created_mask_real)
                binary_mask_real = (real_label_value == additional_id).astype(np.uint8)
                if task_type == "donkey":
                    binary_mask_real = binary_mask_real + (real_label_value == 2).astype(np.uint8)
              
                fake_real_cycle=tf.expand_dims(fake_real_cycle, 0)
                pred_masks =segmentation_model.predict(fake_real_cycle).logits
                created_mask=data_generator_utils.cast_to_int32(data_generator_utils.create_mask(pred_masks))
                created_mask_cyclegan = tf.image.resize(created_mask, [height,width],
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                created_mask_cyclegan = tf.cast(created_mask_cyclegan, np.uint8)
                cyclegan_label_value =vectorized_map_car(created_mask_cyclegan)
                
                binary_mask_cyclegan = (cyclegan_label_value == additional_id).astype(np.uint8)
                if task_type == "donkey":
                    binary_mask_cyclegan = binary_mask_cyclegan + (cyclegan_label_value == 2).astype(np.uint8)
                colored_mask = np.zeros_like(input_image_sim_for_pix2pix)
                colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(binary_mask_cyclegan))
                colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(binary_mask_cyclegan))
                colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(binary_mask_cyclegan))
                png_image = tf.image.encode_png(colored_mask)
                os.makedirs('./'+task_type+'/content/output_plots/cyclegan/'+cyclegan_name+'_additional_mask/', exist_ok=True)
                with open('./'+task_type+'/content/output_plots/cyclegan/'+cyclegan_name+'_additional_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())

                colored_mask = np.zeros_like(input_image_sim_for_pix2pix)
                colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(created_mask_cyclegan))
                colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(created_mask_cyclegan))
                colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(created_mask_cyclegan))
                png_image = tf.image.encode_png(colored_mask)
                os.makedirs('./'+task_type+'/content/output_plots/cyclegan/'+cyclegan_name+'_mask/', exist_ok=True)
                with open('./'+task_type+'/content/output_plots/cyclegan/'+cyclegan_name+'_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())
                
                input_image_sim=tf.expand_dims(input_image_sim, 0)
                pred_masks =segmentation_model.predict(input_image_sim).logits
                created_mask=data_generator_utils.cast_to_int32(data_generator_utils.create_mask(pred_masks))
                created_mask_sim = tf.image.resize(created_mask, [height,width],
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                created_mask_sim = tf.cast(created_mask_sim, np.uint8)

                if task_type=="kitti":
                    vectorized_map=np.vectorize(data_generator_utils.map_values_kitti)
                    
                elif task_type=="donkey":
                    vectorized_map=np.vectorize(data_generator_utils.map_values_donkey)
                
                created_mask_cyclegan=data_generator_utils.cast_to_int32(created_mask_cyclegan)
                created_mask_cyclegan=np.squeeze(created_mask_cyclegan)

                binary_mask_cyclegan=data_generator_utils.cast_to_int32(binary_mask_cyclegan)
                binary_mask_cyclegan=np.squeeze(binary_mask_cyclegan)

                binary_mask_real=data_generator_utils.cast_to_int32(binary_mask_real)
                binary_mask_real=np.squeeze(binary_mask_real)

                created_mask_real=data_generator_utils.cast_to_int32(created_mask_real)
                created_mask_real=np.squeeze(created_mask_real)
                created_mask_sim=data_generator_utils.cast_to_int32(created_mask_sim)
                created_mask_sim=np.squeeze(created_mask_sim)

                error_sim_real=abs(created_mask_sim-created_mask_real)
                error_cyclegan_sim=abs(created_mask_cyclegan-created_mask_sim)
                error_cyclegan_real=abs(created_mask_cyclegan-created_mask_real)
                error_cyclegan_car=abs(binary_mask_cyclegan-binary_mask_real)
    
                error_data_cyclegan = {
                    "sim_real": np.sum(error_sim_real) / (height * width),
                    "sim": np.sum(error_cyclegan_sim) / (height * width),
                    "real": np.sum(error_cyclegan_real) / (height * width),
                    "additional": np.sum(error_cyclegan_car) / (height * width)
                }

                output_folder = './'+task_type+'/content/output_plots/cyclegan/'+cyclegan_name+'_mask_error/'
                os.makedirs(output_folder, exist_ok=True)
                output_path = output_folder + str(dataset_index) + "_" + str(i) +".json"
                with open(output_path, "w") as json_file:
                    json.dump(error_data_cyclegan, json_file, indent=4)

In [20]:
def save_pix2pix_mask_outputs(task_type,limit,segmentation_model,two_classes,generator_pix2pix_mask,pix2pix_mask_name,additional_id,height,width,input_domain,pix2pix_mask_type,dataset_index_list_test,test_indexes_gan,loaded_dictionary_images_real,loaded_dictionary_images_sim,loaded_semantic_id_real,loaded_semantic_id_sim):
    for dataset_index in dataset_index_list_test:
          print(dataset_index)
          counter=0
          for i in test_indexes_gan[dataset_index]:
                if i%50==0:
                    print(i," out of ",len(test_indexes_gan[dataset_index]))
                counter+=1
                input_image_real = np.array(loaded_dictionary_images_real[dataset_index][i])
                input_image_real = tf.convert_to_tensor(input_image_real.astype(np.uint8), np.uint8)
                input_image_real = tf.reverse(input_image_real, axis=[-1])
                
                
                input_image_real = tf.cast(input_image_real, np.float16)
                input_image_real = tf.transpose(input_image_real, (2, 0, 1))
        
                input_image_sim = np.array(loaded_dictionary_images_sim[dataset_index][i])
                input_image_sim_for_pix2pix=input_image_sim
                input_image_sim = tf.convert_to_tensor(input_image_sim.astype(np.uint8), np.uint8)
                input_image_sim = tf.reverse(input_image_sim, axis=[-1])
                
                input_image_sim = tf.cast(input_image_sim, np.float16)
              
                input_image_sim = tf.transpose(input_image_sim, (2, 0, 1))
                
                vectorized_map_car=np.vectorize(data_generator_utils.map_values_car)
        
                colored_mask = np.zeros_like(input_image_sim_for_pix2pix)
                colored_mask2 = np.zeros_like(input_image_sim_for_pix2pix)

                input_image_real=tf.expand_dims(input_image_real, 0)
                pred_masks =segmentation_model.predict(input_image_real).logits
                created_mask=data_generator_utils.cast_to_int32(data_generator_utils.create_mask(pred_masks))
                created_mask_real = tf.image.resize(created_mask, [height,width],
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                created_mask_real = tf.cast(created_mask_real, np.uint8)

                real_label_value =vectorized_map_car(created_mask_real)
                binary_mask_real = (real_label_value == additional_id).astype(np.uint8)
                if task_type == "donkey":
                    binary_mask_real = binary_mask_real + (real_label_value == 2).astype(np.uint8)

                input_image_sim=tf.expand_dims(input_image_sim, 0)
                pred_masks =segmentation_model.predict(input_image_sim).logits
                created_mask=data_generator_utils.cast_to_int32(data_generator_utils.create_mask(pred_masks))
                created_mask_sim = tf.image.resize(created_mask, [height,width],
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                created_mask_sim = tf.cast(created_mask_sim, np.uint8)

                if pix2pix_mask_type=="manual": 
                    if (input_domain=="real"):
                        mask_in=loaded_semantic_id_real[dataset_index][i]
                    else:
                        mask_in=loaded_semantic_id_sim[dataset_index][i]
                    
                else:
                    if (input_domain=="real"):
                        mask_in=created_mask_real[:,:,0]
                    else:
                        mask_in=created_mask_sim[:,:,0]
              
              
                if (input_domain=="real"):
                    colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(mask_in)
                    colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(mask_in)
                    colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(mask_in)
                    input_image_real_for_pix2pix_mask=colored_mask
                    input_image_real_for_pix2pix_mask = tf.convert_to_tensor(input_image_real_for_pix2pix_mask.astype(np.float16), np.float16)
                    input_image_real_for_pix2pix_mask = tf.reverse(input_image_real_for_pix2pix_mask, axis=[-1])
                    input_image_real_for_pix2pix_mask = tf.image.resize(input_image_real_for_pix2pix_mask, [512,512],
                                              method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                    input_image_real_for_pix2pix_mask=tf.expand_dims(input_image_real_for_pix2pix_mask, 0)
                    input_image_real_for_pix2pix_mask = (input_image_real_for_pix2pix_mask / 127.5) - 1
                    fake_real_pix2pix_mask_real = data_generator_utils.generate_images_pix2pix(generator_pix2pix_mask, input_image_real_for_pix2pix_mask,False)
                    fake_real_pix2pix_mask_real = tf.image.resize(fake_real_pix2pix_mask_real, [height,width],
                                      method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                    fake_real_pix2pix_mask_real=fake_real_pix2pix_mask_real * (255/2) + (255/2)
                    fake_real_pix2pix_mask_real = tf.cast(fake_real_pix2pix_mask_real, np.uint8)
                    png_image = tf.image.encode_png(fake_real_pix2pix_mask_real)
                    os.makedirs('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_real/', exist_ok=True)
                    with open('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_real/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                        f.write(png_image.numpy())
                    fake_real_pix2pix_mask_real = tf.cast(fake_real_pix2pix_mask_real, np.float16)
                    fake_real_pix2pix_mask_real = tf.transpose(fake_real_pix2pix_mask_real, (2, 0, 1))


                    fake_real_pix2pix_mask_real=tf.expand_dims(fake_real_pix2pix_mask_real, 0)
                    pred_masks = segmentation_model.predict(fake_real_pix2pix_mask_real).logits
                    created_mask=data_generator_utils.cast_to_int32(data_generator_utils.create_mask(pred_masks))
                    created_mask_pix2pix_mask_real = tf.image.resize(created_mask, [height,width],
                                      method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)


                    pix2pix_label_value=created_mask_pix2pix_mask_real
                    pix2pix_label_value =vectorized_map_car(pix2pix_label_value)

                    
                    
                    binary_mask_pix2pix = (pix2pix_label_value == additional_id).astype(np.uint8)
                    if task_type == "donkey":
                        binary_mask_pix2pix = binary_mask_pix2pix + (pix2pix_label_value == 2).astype(np.uint8)
                    colored_mask = np.zeros_like(input_image_sim_for_pix2pix)
                    colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(binary_mask_pix2pix))
                    colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(binary_mask_pix2pix))
                    colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(binary_mask_pix2pix))
                    png_image = tf.image.encode_png(colored_mask)
                    os.makedirs('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_real_additional_mask/', exist_ok=True)
                    with open('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_real_additional_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                      f.write(png_image.numpy())
                    
                    created_mask_pix2pix_mask_real = tf.cast(created_mask_pix2pix_mask_real, np.uint8)
                    colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(created_mask_pix2pix_mask_real))
                    colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(created_mask_pix2pix_mask_real))
                    colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(created_mask_pix2pix_mask_real))
                    png_image = tf.image.encode_png(colored_mask)
                    os.makedirs('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_real_mask/', exist_ok=True)
                    with open('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_real_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                        f.write(png_image.numpy())
                    created_mask_pix2pix_mask_real=data_generator_utils.cast_to_int32(created_mask_pix2pix_mask_real)
                    created_mask_pix2pix_mask_real=np.squeeze(created_mask_pix2pix_mask_real)
                    

              
                else:
                    colored_mask2[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(mask_in)
                    colored_mask2[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(mask_in)
                    colored_mask2[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(mask_in)
                    input_image_sim_for_pix2pix_mask=colored_mask2
                    input_image_sim_for_pix2pix_mask = tf.convert_to_tensor(input_image_sim_for_pix2pix_mask.astype(np.float16), np.float16)
                    input_image_sim_for_pix2pix_mask = tf.reverse(input_image_sim_for_pix2pix_mask, axis=[-1])
                    input_image_sim_for_pix2pix_mask = tf.image.resize(input_image_sim_for_pix2pix_mask, [512,512],
                                              method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                    input_image_sim_for_pix2pix_mask=tf.expand_dims(input_image_sim_for_pix2pix_mask, 0)
                    input_image_sim_for_pix2pix_mask = (input_image_sim_for_pix2pix_mask / 127.5) - 1  
                    fake_real_pix2pix_mask_sim = data_generator_utils.generate_images_pix2pix(generator_pix2pix_mask, input_image_sim_for_pix2pix_mask,False)
                    fake_real_pix2pix_mask_sim = tf.image.resize(fake_real_pix2pix_mask_sim, [height,width],
                                      method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                    fake_real_pix2pix_mask_sim=fake_real_pix2pix_mask_sim * (255/2) + (255/2)
                    fake_real_pix2pix_mask_sim = tf.cast(fake_real_pix2pix_mask_sim, np.uint8)
                    png_image = tf.image.encode_png(fake_real_pix2pix_mask_sim)
                    os.makedirs('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_sim/', exist_ok=True)
                    with open('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_sim/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                      f.write(png_image.numpy())
                    fake_real_pix2pix_mask_sim = tf.cast(fake_real_pix2pix_mask_sim, np.float16)
                    fake_real_pix2pix_mask_sim = tf.transpose(fake_real_pix2pix_mask_sim, (2, 0, 1))

                    fake_real_pix2pix_mask_sim=tf.expand_dims(fake_real_pix2pix_mask_sim, 0)
                    pred_masks = segmentation_model.predict(fake_real_pix2pix_mask_sim).logits
                    created_mask=data_generator_utils.cast_to_int32(data_generator_utils.create_mask(pred_masks))
                    created_mask_pix2pix_mask_sim = tf.image.resize(created_mask, [height,width],
                                      method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                    
                    
                    pix2pix_label_value=created_mask_pix2pix_mask_sim
                    pix2pix_label_value =vectorized_map_car(pix2pix_label_value)
                    binary_mask_pix2pix = (pix2pix_label_value == additional_id).astype(np.uint8)
                    if task_type == "donkey":
                        binary_mask_pix2pix = binary_mask_pix2pix + (pix2pix_label_value == 2).astype(np.uint8)
                    colored_mask = np.zeros_like(input_image_sim_for_pix2pix)
                    colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(binary_mask_pix2pix))
                    colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(binary_mask_pix2pix))
                    colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(binary_mask_pix2pix))
                    png_image = tf.image.encode_png(colored_mask)
                    os.makedirs('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_sim_additional_mask/', exist_ok=True)
                    with open('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_sim_additional_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                      f.write(png_image.numpy())
                    
                    created_mask_pix2pix_mask_sim = tf.cast(created_mask_pix2pix_mask_sim, np.uint8)
                    colored_mask[:, :, 0] = np.vectorize(data_generator_utils.map_to_b)(tf.squeeze(created_mask_pix2pix_mask_sim))
                    colored_mask[:, :, 1] = np.vectorize(data_generator_utils.map_to_g)(tf.squeeze(created_mask_pix2pix_mask_sim))
                    colored_mask[:, :, 2] = np.vectorize(data_generator_utils.map_to_r)(tf.squeeze(created_mask_pix2pix_mask_sim))
                    png_image = tf.image.encode_png(colored_mask)
                    os.makedirs('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_sim_mask/', exist_ok=True)
                    with open('./'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_sim_mask/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                      f.write(png_image.numpy())
                    created_mask_pix2pix_mask_sim=data_generator_utils.cast_to_int32(created_mask_pix2pix_mask_sim)
                    created_mask_pix2pix_mask_sim=np.squeeze(created_mask_pix2pix_mask_sim)
                    
                
                if task_type=="kitti":
                    vectorized_map=np.vectorize(data_generator_utils.map_values_car)
                elif task_type=="donkey":
                    vectorized_map=np.vectorize(data_generator_utils.map_values_donkey)
                real_label_value=created_mask_real

                
                binary_mask_pix2pix=data_generator_utils.cast_to_int32(binary_mask_pix2pix)
                binary_mask_pix2pix=np.squeeze(binary_mask_pix2pix)
                binary_mask_real=data_generator_utils.cast_to_int32(binary_mask_real)
                binary_mask_real=np.squeeze(binary_mask_real)

                created_mask_real=data_generator_utils.cast_to_int32(created_mask_real)
                created_mask_real=np.squeeze(created_mask_real)
                created_mask_sim=data_generator_utils.cast_to_int32(created_mask_sim)
                created_mask_sim=np.squeeze(created_mask_sim)

                error_pix2pix_car=abs(binary_mask_pix2pix-binary_mask_real)
                error_sim_real=abs(created_mask_sim-created_mask_real)

                if (input_domain=="real"):

                    error_pix2pix_sim_mask_real=abs(created_mask_pix2pix_mask_real-created_mask_sim)
                    error_pix2pix_real_mask_real=abs(created_mask_pix2pix_mask_real-created_mask_real)
                    
                    error_data_pix2pix_mask_real = {
                        "sim_real": np.sum(error_sim_real) / (width * height),
                        "sim": np.sum(error_pix2pix_sim_mask_real) / (width * height),
                        "real": np.sum(error_pix2pix_real_mask_real) / (width * height),
                        "additional": np.sum(error_pix2pix_car) / (width * height)
                    }
    
                    output_folder = './'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_real_mask_error/'
                    os.makedirs(output_folder, exist_ok=True)
                    output_path = output_folder + str(dataset_index) + "_" + str(i) +".json"
                    with open(output_path, "w") as json_file:
                        json.dump(error_data_pix2pix_mask_real, json_file, indent=4)
                    
                else:
                    error_pix2pix_sim_mask_sim=abs(created_mask_pix2pix_mask_sim-created_mask_sim)
                    error_pix2pix_real_mask_sim=abs(created_mask_pix2pix_mask_sim-created_mask_real)
                    error_data_pix2pix_mask_sim = {
                        "sim_real": np.sum(error_sim_real) / (width * height),
                        "sim": np.sum(error_pix2pix_sim_mask_sim) / (width * height),
                        "real": np.sum(error_pix2pix_real_mask_sim) / (width * height),
                        "additional": np.sum(error_pix2pix_car) / (width * height)
                    }
                    output_folder = './'+task_type+'/content/output_plots/pix2pix_mask_'+pix2pix_mask_type+'/'+pix2pix_mask_name+'_sim_mask_error/'
                    os.makedirs(output_folder, exist_ok=True)
                    output_path = output_folder + str(dataset_index) + "_" + str(i) +".json"
                    with open(output_path, "w") as json_file:
                        json.dump(error_data_pix2pix_mask_sim, json_file, indent=4)
                    

In [5]:
task_type='donkey'
output_folder = './'+task_type+'/content/output_plots'
plot=False
two_classes=False
limit=10000

In [6]:
file_path = './'+task_type+'/content/datasets/h5_out/raw_image_sim.h5'
loaded_dictionary_images_sim=data_generator_utils.load_h5_to_dictionary(file_path)
file_path = './'+task_type+'/content/datasets/h5_out/raw_image_real.h5'
loaded_dictionary_images_real=data_generator_utils.load_h5_to_dictionary(file_path)
file_path = './'+task_type+'/content/datasets/h5_out/semantic_id_list_real.h5'
loaded_semantic_id_real=data_generator_utils.load_h5_to_dictionary(file_path)
file_path = './'+task_type+'/content/datasets/h5_out/semantic_id_list_sim.h5'
loaded_semantic_id_sim=data_generator_utils.load_h5_to_dictionary(file_path)

if task_type=="kitti":
    file_path = './'+task_type+'/content/datasets/h5_out/bounding_boxes_sim.h5'
    loaded_bounding_sim=data_generator_utils.load_h5_to_dictionary(file_path)
    file_path = './'+task_type+'/content/datasets/h5_out/bounding_boxes_real.h5'
    loaded_bounding_real=data_generator_utils.load_h5_to_dictionary(file_path)
    height,width=374,1238
    road=0
    additional_id=3
    additional_id_init=13
    dataset_index_list_test=["0001","0002","0006","0018","0020"]
    pattern = 'tvvttvttnn'
    
elif task_type=="donkey":
    height,width=140,320
    road=1
    additional_id=1
    additional_id_init=2
    dataset_index_list_test=["0001"]
    pattern = 'vvvvvvvvvv'
    
else:
    print("Choose a set")

In [7]:
train_indexes_gan,test_indexes_gan=data_generator_utils.get_gan_indexes(dataset_index_list_test,loaded_dictionary_images_real,loaded_dictionary_images_sim,pattern)
loaded_dictionary_images_real,loaded_dictionary_images_sim,loaded_semantic_id_real,loaded_semantic_id_sim=data_generator_utils.crop_data_dictionaries(task_type,dataset_index_list_test,loaded_dictionary_images_real,loaded_dictionary_images_sim,loaded_semantic_id_real,loaded_semantic_id_sim)

GAN train and test
Dataset 0001
Train:  0
Test:  196
140 320


In [8]:
checkpoint_file_path = "./"+task_type+"/content/segmentation_checkpoints/Model_weights.hdf5"
segmentation_model=data_generator_utils.load_segmentation_model(checkpoint_file_path,task_type)

3
Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-10-27 16:20:13.173039: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-27 16:20:13.173192: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the check

In [ ]:
real_path_name="real"
sim_path_name="sim"
save_sim_real_outputs(task_type,segmentation_model,real_path_name,sim_path_name,additional_id,height,width,dataset_index_list_test,test_indexes_gan,loaded_dictionary_images_real,loaded_dictionary_images_sim)

In [ ]:
checkpoint_paths = ['./'+task_type+'/content/gan_checkpoints/cyclegan_checkpoints/1','./'+task_type+'/content/gan_checkpoints/cyclegan_checkpoints/2','./'+task_type+'/content/gan_checkpoints/cyclegan_checkpoints/3']
checkpoint_names=["cyclegan_1","cyclegan_2","cyclegan_3"]

for checkpoint_path,checkpoint_name in zip(checkpoint_paths,checkpoint_names):
    print(checkpoint_path)
    generator_cyclegan = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
    ckpt_cyclegan = tf.train.Checkpoint(generator_g=generator_cyclegan)
    ckpt_manager_cyclegan = tf.train.CheckpointManager(ckpt_cyclegan, checkpoint_path, max_to_keep=5)
    if ckpt_manager_cyclegan.latest_checkpoint:
        ckpt_cyclegan.restore(ckpt_manager_cyclegan.latest_checkpoint)
        print ('Checkpoint restored')
    else:
        print ('No Checkpoint!!!')
    print(height,width)
    save_cyclegan_outputs(task_type,segmentation_model,generator_cyclegan,checkpoint_name,additional_id,height,width,dataset_index_list_test,test_indexes_gan,loaded_dictionary_images_real,loaded_dictionary_images_sim)

In [ ]:
checkpoint_paths = ['./'+task_type+'/content/gan_checkpoints/pix2pix_checkpoints/1','./'+task_type+'/content/gan_checkpoints/cyclegan_checkpoints/2','./'+task_type+'/content/gan_checkpoints/cyclegan_checkpoints/3']
checkpoint_names=["pix2pix_mask_1","pix2pix_mask_2","pix2pix_mask_3"]
    

input_domain="sim"
pix2pix_mask_type="manual"

for checkpoint_path,checkpoint_name in zip(checkpoint_paths,checkpoint_names):
    generator_pix2pix_mask_real = data_generator_utils.Generator()
    ckpt_pix2pix_mask_real = tf.train.Checkpoint(
                                     generator=generator_pix2pix_mask_real)
    ckpt_manager_pix2pix_mask_real = tf.train.CheckpointManager(ckpt_pix2pix_mask_real, checkpoint_path, max_to_keep=5)
    if ckpt_manager_pix2pix_mask_real.latest_checkpoint:
        ckpt_pix2pix_mask_real.restore(ckpt_manager_pix2pix_mask_real.latest_checkpoint)
        print ('Checkpoint restored')
    else:
        print ('No Checkpoint! Check source path')
    save_pix2pix_mask_outputs(task_type,limit,segmentation_model,two_classes,generator_pix2pix_mask_real,checkpoint_name,additional_id,height,width,input_domain,pix2pix_mask_type,dataset_index_list_test, test_indexes_gan, loaded_dictionary_images_real,loaded_dictionary_images_sim,loaded_semantic_id_real,loaded_semantic_id_sim)